In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as scs
import matplotlib.pyplot as plt
import scipy.optimize as sco


In [2]:
df = pd.read_csv('16Bitcoin.csv',index_col = 'Date',usecols = [0,1])

In [ ]:
df2 = pd.read_csv('16GOLD.csv',index_col = 'Date',usecols = [0,1])

In [ ]:
df3=pd.merge(df,df2,how='left',left_on='Date',right_on='Date')

In [ ]:
returns =np.log(df3 / df3.shift(1))
print (returns)
print("每日收益平均值")
print (returns.mean())
print (32*"-")
print("年化收益")
print (returns.mean()*365)
print (32*"-")
print("相关系数")
print (returns.corr())#计算相关系数
print (32*"-")
print ("协方差")
print (returns.cov()*365)
print (32*"-")
returns.hist(bins = 50, figsize = (12,8))

In [ ]:
weights = np.random.random(2)
weights /= np.sum(weights)
print ('初始化权重',weights)
#计算预期组合年化收益、组合方差和组合标准差
print ('初始权重收益',np.sum(returns.mean()*weights)*365)
print ('初始权重组合方差',np.dot(weights.T, np.dot(returns.cov()*365,weights)))
print ('初始权重组合标准差',np.sqrt(np.dot(weights.T, np.dot(returns.cov()* 365,weights))))

In [ ]:
port_returns = []
port_variance = []
for p in range(4000):
    weights = np.random.random(2)
    weights /=np.sum(weights)
    port_returns.append(np.sum(returns.mean()*364*weights))
    port_variance.append(np.sqrt(np.dot(weights.T, np.dot(returns.cov()*364, weights))))
port_returns = np.array(port_returns)
port_variance = np.array(port_variance)

In [ ]:
#投资组合优化1——sharpe最大
#建立statistics函数来记录重要的投资组合统计数据（收益，方差和夏普比）
#通过对约束最优问题的求解，得到最优解。其中约束是权重总和为1。
print (32*"-")
print ("投资组合优化方法1--sharpe值最大")
def statistics(weights):
    weights = np.array(weights)
    port_returns = np.sum(returns.mean()*weights)*365
    port_variance = np.sqrt(np.dot(weights.T, np.dot(returns.cov()*365,weights)))
    return np.array([port_returns, port_variance, port_returns/port_variance])
#最优化投资组合的推导是一个约束最优化问题
#最小化夏普指数的负值
def min_sharpe(weights):
    return -statistics(weights)[2]
#约束是所有参数(权重)的总和为1。这可以用minimize函数的约定表达如下
cons = ({'type':'eq', 'fun':lambda x: np.sum(x)-1})
#我们还将参数值(权重)限制在0和1之间。这些值以多个元组组成的一个元组形式提供给最小化函数
bnds = tuple((0,1) for x in range(2))
#优化函数调用中忽略的唯一输入是起始参数列表(对权重的初始猜测)。我们简单的使用平均分布。
opts = sco.minimize(min_sharpe, 2*[1./2,], method = 'SLSQP', bounds = bnds, constraints = cons)
print (opts)
print ("权重",opts['x'].round(3))
#预期收益率、预期波动率、最优夏普指数
print ('最大sharpe指数预期收益' , statistics(opts['x']).round(3))
print (32*"-")

In [ ]:
print ("投资组合优化方法2--方差最小")
#投资组合优化2——方差最小
#接下来，我们通过方差最小来选出最优投资组合。
#我们定义一个函数对 方差进行最小化
def min_variance(weights):
    return statistics(weights)[1]
optv = sco.minimize(min_variance, 2*[1./2,],method = 'SLSQP', bounds = bnds, constraints = cons)
print (optv)
print ("权重",optv['x'].round(3))
#得到的预期收益率、波动率和夏普指数
print ('最小方差预期收益' , statistics(optv['x']).round(3))
#组合的有效前沿
#有效前沿有既定的目标收益率下方差最小的投资组合构成。
#在不同目标收益率水平（target_returns）循环时，最小化的一个约束条件会变化。
target_returns = np.linspace(0.0,0.5,50)
target_variance = []
for tar in target_returns:
    cons = ({'type':'eq','fun':lambda x:statistics(x)[0]-tar},{'type':'eq','fun':lambda x:np.sum(x)-1})
    res = sco.minimize(min_variance, 2*[1./2,],method = 'SLSQP', bounds = bnds, constraints = cons)
    target_variance.append(res['fun'])
target_variance = np.array(target_variance)